In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
import glob
%matplotlib inline

In [2]:
"""
The Camera class is used to calculate calibration parameters
based on a set of calibration images passed to the calibrate()
method. This class can be used dewarp an input image passed
as an np blob or as a file name, with the undistort() method. 
"""
class Camera():
    def __init__(self, verbose=False):
        self.mtx = None
        self.rvecs = None
        self.tvecs = None
        self.shape = None
        self.dist = None
        self.verbose = verbose
        
    def calibrate(self, calibration_folder):
        images = glob.glob("%s/calibration*.jpg" % calibration_folder)
        print('performing calibration on %d images in %s...' % (len(images), calibration_folder))
        
        objpoints = []
        imgpoints = []
        nx = 9
        ny = 6
        objp = np.zeros((ny*nx, 3), np.float32)
        objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

        for i in images:
            if (self.verbose):
                print("reading calibration file: %s" % i)
            raw_img = mpimg.imread(i)
            gray_img = cv2.cvtColor(raw_img, cv2.COLOR_RGB2GRAY)
            ret, corners = cv2.findChessboardCorners(gray_img, (nx, ny), None)
            self.shape = gray_img.shape[::-1]
            if (ret):
                imgpoints.append(corners)
                objpoints.append(objp)
                
            elif (self.verbose):
                print("WARNING:: wrong # of corners found in : ", i)

        ret, self.mtx, self.dist, self.rvecs, self.tvecs = cv2.calibrateCamera(objpoints,
                                                                          imgpoints, self.shape, None, None)
        print("calibration complete")
        if (self.verbose):
            print("mtx: ", self.mtx)
            print("dist:", self.dist)
            print("rvecs: ", self.rvecs)
            print("tvecs: ", self.tvecs)
                
    def test_calibration(self, img_file):
        img = cv2.imread(img_file)
        dst = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        plt.imshow(dst)
    
    def undistort(self, img):
        if (isinstance(img, np.ndarray)):
            return cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        elif (isinstance(img, str)):
            raw = cv2.imread(img)
            return cv2.undistort(raw, self.mtx, self.dist, None, self.mtx)
        else:
            raise TypeError("input argument is invalid type", type(img))


camera = Camera(verbose=False)
camera.calibrate("camera_cal")

performing calibration on 20 images in camera_cal...
calibration complete


In [38]:
"""
The SobelFilter class is used process a raw image
and return a processed binary image to extract 
the lane lines
"""
class SobelFilter():
    def __init__(self, verbose=False):
        self.verbose=verbose
        self.max = 255

    def grayscale(self, img):
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    def hls_select(self, img, thresh=(140, 255)):
        hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]

        binary = np.zeros_like(img)
        binary[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = self.max
        
        return binary
        
    def magnitude_threshold(self, img, sobel_kernel=5, threshold=(35, 100)):
        gray = self.grayscale(img)
        x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        magnitude = np.sqrt(x*x + y*y)
        scaled = np.uint8(255* magnitude / np.max(magnitude))
        binary = np.zeros_like(img)
        binary[(scaled >= threshold[0]) & (scaled <= threshold[1])] = self.max
        
        return binary
    
    def absolute_threshold(self, img, orient='x', threshold=(35, 100)):
        gray = self.grayscale(img)
        if orient == 'x':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        if orient == 'y':
            sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
        scaled = np.uint8(255*sobel / np.max(sobel))
        binary = np.zeros_like(img)
        binary[(scaled >= threshold[0]) & (scaled <= threshold[1])] = self.max
        
        return binary
        
    def direction_threshold(self, img, sobel_kernel=15, thresh=(0.7, 1.3)):
        gray = self.grayscale(img)
        x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
        y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
        direction = np.arctan2(np.absolute(y), np.absolute(x))
        binary = np.zeros_like(img)
        binary[(direction >= thresh[0]) & (direction <= thresh[1])] = self.max        

        return binary
    
    def process(self, img):
        x = self.absolute_threshold(img, 'x')
        y = self.absolute_threshold(img, 'y')
        magnitude = self.magnitude_threshold(img)
        direction = self.direction_threshold(img)        
        s = self.hls_select(img)
        
        output = np.zeros_like(img)
        output[((x == self.max) & (y == self.max)) |
               ((magnitude == self.max) & (direction == self.max)) |
               (s == self.max)] = self.max
        
        return output

In [39]:
"""
The LaneDetector class will detect lanes from images performing the following:
 1) Applies a distortion correction to raw images.
 2) Uses a set of Sobel filters to threshold the relevant lane line pixels
 3) Applies a perspective transform to rectify binary image ("birds-eye view").
 4) Detects lane pixels and fit to find the lane boundary.
 5) Determines the curvature of the lane and vehicle position with respect to center.
 6) Warps the detected lane boundaries back onto the original image.
 7) Outputs visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
"""        
class LaneDetector():
    def __init__(self, verbose=False):
        self.verbose = verbose
        self.sobel = SobelFilter()
        self.output = None
        
    def threshold(self, img):        
        self.output = self.sobel.process(img)
    
    def transform(self, img):
        return None
    
    def detect_lanes(self, img):
        return None
    
    def process(self, img):
        self.threshold(img)
        self.transform(img)
        self.detect_lanes(img)
        return self.output

lane_detector = LaneDetector(verbose=False)

In [40]:
def save(image, name, directory):
    cv2.imwrite(directory + "/" + i, image)

test_images = os.listdir("test_images/")

for i in test_images:
    if (i[-3:] == 'jpg' or i[-3:] == 'png'):
        img = mpimg.imread('test_images/%s' % i)
        undistored_image = camera.undistort(img)
        output = lane_detector.process(undistored_image)
        save(output, i, "output_images")

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [7]:
def process_image(image):
    # Copy the image argument frame locally, 
    # otherwise, the full video will be a single frame
    img = image

    # Pass the raw image through the camera claass
    # to find the undistored version
    undistored_image = camera.undistort(img)
    
    # Pass the undistored image through the lane
    # detector to find the post processed version
    output = lane_detector.process(undistored_image)

    # Returns the process imaged back to the 
    # video processor
    return output

In [8]:
output_video = 'output_images/project_video_output.mp4'
input_video = VideoFileClip('project_video.mp4')
clip = input_video.fl_image(process_image)
%time clip.write_videofile(output_video, audio=False)

[MoviePy] >>>> Building video output_images/project_video_output.mp4
[MoviePy] Writing video output_images/project_video_output.mp4


100%|█████████▉| 1260/1261 [03:47<00:00,  5.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video_output.mp4 

CPU times: user 4min 51s, sys: 8.68 s, total: 5min
Wall time: 3min 48s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_video))